# Digital Image Processing - Project Overview
## Submitted By: AZAN LATIF
## ROll NO: 2023-SE-14


>This project focuses on developing an automated system for melanoma detection from dermoscopic images. The process is broken down into several key stages:

### 1. Lesion Segmentation & Mask Generation
This initial phase involves pre-processing dermoscopic images to isolate the lesion area. It includes:
*   **Image Enhancement**: Improving image quality through techniques like CLAHE to adjust contrast.
*   **Hair Removal**: Eliminating hair artifacts using morphological operations (e.g., black-hat transform) and inpainting to ensure cleaner lesion boundaries.
*   **Lesion Segmentation**: Applying techniques like Otsu's thresholding, morphological closing, and flood fill to accurately delineate the lesion from the surrounding skin, generating a binary mask for each lesion.

### 2. Feature Extraction
Once the lesions are segmented, relevant features are extracted from the masked areas of the original images. These features are typically quantitative measurements that describe the lesion's characteristics, such as color (mean and standard deviation for BGR channels) and potentially texture or shape descriptors.

### 3. Classification
Extracted features are then used to train a machine learning classifier (e.g., Random Forest). The classifier learns patterns from these features to distinguish between benign lesions and melanoma.

### 4. Evaluation
Both the segmentation and classification steps are rigorously evaluated using appropriate metrics:
*   **Segmentation Evaluation**: Assessing the accuracy of generated masks against ground truth masks (e.g., using confusion matrix metrics like Accuracy, Sensitivity, and Specificity).
*   **Classification Evaluation**: Measuring the performance of the trained classifier (e.g., Accuracy, Precision, Recall, F1-score) on a held-out test set.

It seems the issue is due to a data imbalance, where your dataset `y` (and consequently `y_train` and `y_test`) contains only one class. This causes the classification metrics to be 1.0 because the model is only predicting that single class, which is always correct within the given test set. Let's verify the class distribution:

In [41]:
import numpy as np
import pandas as pd

print("Value counts for all labels (y):\n", pd.Series(y).value_counts())
print("\nValue counts for training labels (y_train):\n", pd.Series(y_train).value_counts())
print("\nValue counts for test labels (y_test):\n", pd.Series(y_test).value_counts())

Value counts for all labels (y):
 0    167
1     33
Name: count, dtype: int64

Value counts for training labels (y_train):
 0    134
1     26
Name: count, dtype: int64

Value counts for test labels (y_test):
 0    33
1     7
Name: count, dtype: int64


In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
BASE_PATH = "/content/drive/MyDrive/PH_2_PROJECT/PH2Dataset"

In [24]:
import os
print(os.listdir(BASE_PATH)[:5])

['PH2_dataset.xlsx', 'PH2_dataset.txt', 'Readme.txt', 'PH2 Dataset images', 'generated_masks']


PROJECT START

Task 1 — Lesion Segmentation & Mask Generation

🔹 Required Libraries

In [25]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from tqdm import tqdm

🔹 Helper Functions (Image Processing Pipeline) 1️⃣ Image Enhancement

In [26]:
def enhance_image(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    l = clahe.apply(l)
    enhanced = cv2.merge((l, a, b))
    return cv2.cvtColor(enhanced, cv2.COLOR_LAB2BGR)

2️⃣ Hair Removal (Black-hat + Inpainting)

In [27]:
def remove_hairs(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (17,17))
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)
    _, mask = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)
    return cv2.inpaint(img, mask, 1, cv2.INPAINT_TELEA)

3️⃣ Lesion Segmentation

In [28]:
def segment_lesion(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)

    _, binary = cv2.threshold(
    blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
)


    # 1️⃣ CLOSE — lesion boundaries connect karne ke liye
    kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7))
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel_close)

    # 2️⃣ FILL HOLES — gaps remove karne ke liye
    h, w = binary.shape
    floodfill = binary.copy()
    mask = np.zeros((h+2, w+2), np.uint8)
    cv2.floodFill(floodfill, mask, (0,0), 255)
    floodfill_inv = cv2.bitwise_not(floodfill)
    filled = binary | floodfill_inv

    return filled

🔹 Main Mask Generation Loop

In [29]:
BASE_PATH = "/content/drive/MyDrive/PH_2_PROJECT/PH2Dataset/PH2 Dataset images"
SAVE_PATH = "/content/drive/MyDrive/PH_2_PROJECT/PH2Dataset/generated_masks"

os.makedirs(SAVE_PATH, exist_ok=True)
os.makedirs(SAVE_PATH, exist_ok=True)

y_true_all, y_pred_all = [], []

for folder in tqdm(os.listdir(BASE_PATH)):
    folder_path = os.path.join(BASE_PATH, folder)

    if not os.path.isdir(folder_path):
        continue

    img_path = os.path.join(
        folder_path, f"{folder}_Dermoscopic_Image"
    )
    gt_path = os.path.join(
        folder_path, f"{folder}_lesion"
    )

    img_file = os.listdir(img_path)[0]
    gt_file  = os.listdir(gt_path)[0]

    img = cv2.imread(os.path.join(img_path, img_file))
    gt  = cv2.imread(os.path.join(gt_path, gt_file), 0)

    if img is None or gt is None:
        continue

    img = enhance_image(img)
    img = remove_hairs(img)
    pred_mask = segment_lesion(img)

    cv2.imwrite(
        os.path.join(SAVE_PATH, f"{folder}_mask.png"),
        pred_mask
    )

    y_true_all.extend((gt > 0).flatten())
    y_pred_all.extend((pred_mask > 0).flatten())

100%|██████████| 200/200 [01:08<00:00,  2.91it/s]


In [30]:
import os

path_to_verify = '/content/drive/MyDrive/PH_2_PROJECT/PH2Dataset/PH2 Dataset images'

if os.path.exists(path_to_verify):
    if os.path.isdir(path_to_verify):
        print(f"Path '{path_to_verify}' exists and is a directory.")
        print("Contents (first 10 items):", os.listdir(path_to_verify)[:5])
    else:
        print(f"Path '{path_to_verify}' exists but is not a directory.")
else:
    print(f"Path '{path_to_verify}' does not exist.")

Path '/content/drive/MyDrive/PH_2_PROJECT/PH2Dataset/PH2 Dataset images' exists and is a directory.
Contents (first 10 items): ['IMD002', 'IMD003', 'IMD004', 'IMD008', 'IMD006']


📊 3. Segmentation Evaluation (Dataset-Level)

In [31]:
cm = confusion_matrix(y_true_all, y_pred_all)
TN, FP, FN, TP = cm.ravel()

accuracy = (TP + TN) / (TP + TN + FP + FN)
sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)

print("Confusion Matrix:\n", cm)
print(f"Accuracy     : {accuracy:.4f}")
print(f"Sensitivity  : {sensitivity:.4f}")
print(f"Specificity  : {specificity:.4f}")

Confusion Matrix:
 [[ 2133754 57593928]
 [  933741 27501900]]
Accuracy     : 0.3361
Sensitivity  : 0.9672
Specificity  : 0.0357


Step 1: Import libraries and mount drive

In [32]:
from google.colab import drive
import os
import cv2
import numpy as np
import pandas as pd

drive.mount('/content/drive')

BASE_PATH = "/content/drive/MyDrive/PH_2_PROJECT/PH2Dataset/PH2 Dataset images"
SAVE_PATH = "/content/drive/MyDrive/PH_2_PROJECT/PH2Dataset/Masks"  # jahan masks saved hain
LABEL_TXT = "/content/drive/MyDrive/PH_2_PROJECT/PH2Dataset/PH2_dataset.txt"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load labels from .txt file

In [33]:
label_dict = {}

with open(LABEL_TXT, "r") as f:
    lines = f.readlines()

for line in lines:
    if line.strip() == "":
        continue

    parts = line.strip().split("||")
    parts = [p.strip() for p in parts if p.strip() != ""]

    # Skip header
    if parts[0] == "Name":
        continue

    img_id = parts[0]

    try:
        melanoma_val = parts[-1].split()[0]
        melanoma_val = int(melanoma_val)
        label = 1 if melanoma_val > 0 else 0
        label_dict[img_id] = label
    except:
        continue

print("Total labels loaded:", len(label_dict))
print("Sample:", list(label_dict.items())[:10])


Total labels loaded: 212
Sample: [('IMD003', 1), ('IMD009', 1), ('IMD016', 1), ('IMD022', 1), ('IMD024', 1), ('IMD025', 1), ('IMD035', 1), ('IMD038', 1), ('IMD042', 1), ('IMD044', 1)]


 Define feature extraction function

In [34]:
def extract_features(img, mask):
    features = []

    # strict binary mask
    mask = (mask > 0).astype(np.uint8)

    for i in range(3):  # BGR channels
        channel = img[:, :, i]
        lesion_pixels = channel[mask == 1]

        if lesion_pixels.size < 50:   # too small region
            features.extend([0, 0])
        else:
            features.extend([
                np.mean(lesion_pixels) / 255.0,
                np.std(lesion_pixels) / 255.0
            ])
    return features



Load images, masks, extract features

In [35]:
from google.colab import drive
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# ----------------------
# Step 0: Mount Drive
# ----------------------
drive.mount('/content/drive')

# ----------------------
# Step 1: Define Paths
# ----------------------
BASE_PATH = "/content/drive/MyDrive/PH_2_PROJECT/PH2Dataset/PH2 Dataset images"
SAVE_PATH = "/content/drive/MyDrive/PH_2_PROJECT/PH2Dataset/generated_masks"
LABEL_TXT = "/content/drive/MyDrive/PH_2_PROJECT/PH2Dataset/PH2_dataset.txt"

# ----------------------
# Step 2: Load Labels from .txt
# ----------------------
label_dict = {}

with open(LABEL_TXT, "r") as f:
    lines = f.readlines()

for line in lines:
    if line.strip() == "":
        continue

    parts = line.strip().split("||")
    parts = [p.strip() for p in parts if p.strip() != ""]

    # Skip header
    if parts[0] == "Name":
        continue

    img_id = parts[0]

    try:
        melanoma_val = parts[-1].split()[0]
        melanoma_val = int(melanoma_val)
        label = 1 if melanoma_val > 0 else 0
        label_dict[img_id] = label
    except:
        continue

print("Total labels loaded:", len(label_dict))
print("Sample labels:", list(label_dict.items())[:10])

# ----------------------
# Step 3: Define Feature Extraction
# ----------------------
def extract_features(img, mask):
    """
    Extract simple features from masked image:
    mean and std for each channel (BGR)
    """
    features = []

    # Ensure mask is binary
    _, mask_bin = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

    for i in range(3):  # B, G, R channels
        masked_pixels = img[:, :, i][mask_bin > 0]
        if masked_pixels.size == 0:
            features.extend([0, 0])
        else:
            features.extend([masked_pixels.mean(), masked_pixels.std()])
    return features  # total 6 features

# ----------------------
# Step 4: Load Images, Masks and Extract Features
# ----------------------
X = []
y = []

missing_masks = []
missing_images = []

for folder in os.listdir(BASE_PATH):
    folder_path = os.path.join(BASE_PATH, folder)

    if folder not in label_dict:
        continue

    # Mask path
    mask_path = os.path.join(SAVE_PATH, f"{folder}_mask.png")
    if not os.path.exists(mask_path):
        missing_masks.append(folder)
        continue

    # Image path
    img_folder = os.path.join(folder_path, f"{folder}_Dermoscopic_Image")
    if not os.path.exists(img_folder):
        missing_images.append(folder)
        continue

    img_files = os.listdir(img_folder)
    if len(img_files) == 0:
        missing_images.append(folder)
        continue

    img_file = img_files[0]
    img = cv2.imread(os.path.join(img_folder, img_file))
    mask = cv2.imread(mask_path, 0)  # grayscale

    if img is None or mask is None:
        missing_images.append(folder)
        continue

    feats = extract_features(img, mask)
    X.append(feats)
    y.append(label_dict[folder])

X = np.array(X)
y = np.array(y)

print("Features shape:", X.shape)
print("Labels shape:", y.shape)
print("Missing masks:", len(missing_masks))
print("Missing images:", len(missing_images))



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total labels loaded: 212
Sample labels: [('IMD003', 1), ('IMD009', 1), ('IMD016', 1), ('IMD022', 1), ('IMD024', 1), ('IMD025', 1), ('IMD035', 1), ('IMD038', 1), ('IMD042', 1), ('IMD044', 1)]
Features shape: (200, 6)
Labels shape: (200,)
Missing masks: 0
Missing images: 0


Train/Test Split Example

In [36]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

Classifier:

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Train Random Forest
clf = RandomForestClassifier(
    n_estimators=50,
    max_depth=5,
    min_samples_split=5,
    random_state=42
)
clf.fit(X_train, y_train)

# Predict
y_pred = clf.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        40

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40



# Task
Update the label extraction logic in cell `cM3_QiNhYYbp` to correctly parse the 'Clinical Diagnosis' column from `PH2_dataset.txt`, mapping '2' to '1' (Melanoma) and all other values to '0' (Benign). Then, run the modified cell to reload features and labels. Subsequently, re-run the train-test split, print the value counts for `y`, `y_train`, and `y_test` to confirm the new label distribution, and finally, re-train and re-evaluate the Random Forest classifier to assess its performance with the corrected labels.

## Correct Label Extraction

### Subtask:
Modify the existing code in cell `cM3_QiNhYYbp` to correctly extract labels from the `PH2_dataset.txt` file, mapping '2' from the 'Clinical Diagnosis' column to '1' (Melanoma) and all other values to '0' (Benign).


**Reasoning**:
The user wants to modify the label extraction logic in cell `cM3_QiNhYYbp` to correctly map 'Clinical Diagnosis' values to binary labels. I will provide the updated code for that cell.



In [38]:
from google.colab import drive
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# ----------------------
# Step 0: Mount Drive
# ----------------------
drive.mount('/content/drive')

# ----------------------
# Step 1: Define Paths
# ----------------------
BASE_PATH = "/content/drive/MyDrive/PH_2_PROJECT/PH2Dataset/PH2 Dataset images"
SAVE_PATH = "/content/drive/MyDrive/PH_2_PROJECT/PH2Dataset/generated_masks"
LABEL_TXT = "/content/drive/MyDrive/PH_2_PROJECT/PH2Dataset/PH2_dataset.txt"

# ----------------------
# Step 2: Load Labels from .txt
# ----------------------
label_dict = {}

with open(LABEL_TXT, "r") as f:
    lines = f.readlines()

for line in lines:
    if line.strip() == "":
        continue

    parts = line.strip().split("||")
    parts = [p.strip() for p in parts if p.strip() != ""]

    # Skip header
    if parts[0] == "Name":
        continue

    img_id = parts[0]

    try:
        clinical_diagnosis_str = parts[2].strip() # Extract 'Clinical Diagnosis' column
        label = 1 if clinical_diagnosis_str == '2' else 0 # Map '2' to 1 (Melanoma), others to 0 (Benign)
        label_dict[img_id] = label
    except IndexError:
        print(f"Warning: Clinical Diagnosis column not found for {img_id}, skipping.")
        continue
    except Exception as e:
        print(f"Warning: Error processing clinical diagnosis for {img_id}: {e}, skipping.")
        continue

print("Total labels loaded:", len(label_dict))
print("Sample labels:", list(label_dict.items())[:10])

# ----------------------
# Step 3: Define Feature Extraction
# ----------------------
def extract_features(img, mask):
    """
    Extract simple features from masked image:
    mean and std for each channel (BGR)
    """
    features = []

    # Ensure mask is binary
    _, mask_bin = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

    for i in range(3):  # B, G, R channels
        masked_pixels = img[:, :, i][mask_bin > 0]
        if masked_pixels.size == 0:
            features.extend([0, 0])
        else:
            features.extend([masked_pixels.mean(), masked_pixels.std()])
    return features  # total 6 features

# ----------------------
# Step 4: Load Images, Masks and Extract Features
# ----------------------
X = []
y = []

missing_masks = []
missing_images = []

for folder in os.listdir(BASE_PATH):
    folder_path = os.path.join(BASE_PATH, folder)

    if folder not in label_dict:
        continue

    # Mask path
    mask_path = os.path.join(SAVE_PATH, f"{folder}_mask.png")
    if not os.path.exists(mask_path):
        missing_masks.append(folder)
        continue

    # Image path
    img_folder = os.path.join(folder_path, f"{folder}_Dermoscopic_Image")
    if not os.path.exists(img_folder):
        missing_images.append(folder)
        continue

    img_files = os.listdir(img_folder)
    if len(img_files) == 0:
        missing_images.append(folder)
        continue

    img_file = img_files[0]
    img = cv2.imread(os.path.join(img_folder, img_file))
    mask = cv2.imread(mask_path, 0)  # grayscale

    if img is None or mask is None:
        missing_images.append(folder)
        continue

    feats = extract_features(img, mask)
    X.append(feats)
    y.append(label_dict[folder])

X = np.array(X)
y = np.array(y)

print("Features shape:", X.shape)
print("Labels shape:", y.shape)
print("Missing masks:", len(missing_masks))
print("Missing images:", len(missing_images))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total labels loaded: 200
Sample labels: [('IMD003', 0), ('IMD009', 0), ('IMD016', 0), ('IMD022', 0), ('IMD024', 0), ('IMD025', 0), ('IMD035', 0), ('IMD038', 0), ('IMD042', 0), ('IMD044', 0)]
Features shape: (200, 6)
Labels shape: (200,)
Missing masks: 0
Missing images: 0


**Reasoning**:
Now that the labels `y` have been reloaded with the correct logic, I need to re-run the train-test split to ensure `y_train` and `y_test` reflect the new label distribution. Then, I will print the value counts for `y`, `y_train`, and `y_test` to confirm the class distribution.



In [39]:
from sklearn.model_selection import train_test_split
import pandas as pd # Import pandas for value_counts

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Value counts for all labels (y):\n", pd.Series(y).value_counts())
print("\nValue counts for training labels (y_train):\n", pd.Series(y_train).value_counts())
print("\nValue counts for test labels (y_test):\n", pd.Series(y_test).value_counts())

Value counts for all labels (y):
 0    167
1     33
Name: count, dtype: int64

Value counts for training labels (y_train):
 0    134
1     26
Name: count, dtype: int64

Value counts for test labels (y_test):
 0    33
1     7
Name: count, dtype: int64


**Reasoning**:
The labels have been corrected and the data split. The next logical step is to re-train the Random Forest classifier with the updated `X_train` and `y_train` and then evaluate its performance on the `X_test` and `y_test` to reflect the new label distribution.



In [40]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Train Random Forest
clf = RandomForestClassifier(
    n_estimators=50,
    max_depth=5,
    min_samples_split=5,
    random_state=42
)
clf.fit(X_train, y_train)

# Predict
y_pred = clf.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.85
              precision    recall  f1-score   support

           0       0.85      1.00      0.92        33
           1       1.00      0.14      0.25         7

    accuracy                           0.85        40
   macro avg       0.92      0.57      0.58        40
weighted avg       0.87      0.85      0.80        40



## Final Task

### Subtask:
Summarize the changes made, the new label distribution, and the resulting performance metrics of the classifier. If significant class imbalance still exists and affects performance, I will suggest further steps.


## Summary:

### Q&A
1.  **Summarize the changes made:**
    The label extraction logic in cell `cM3_QiNhYYbp` was updated to correctly parse the 'Clinical Diagnosis' column from `PH2_dataset.txt`. Specifically, values of '2' were mapped to '1' (Melanoma), and all other values were mapped to '0' (Benign). Following this, features and labels were reloaded, the train-test split was re-executed with stratification, and the Random Forest classifier was re-trained and re-evaluated.
2.  **Summarize the new label distribution:**
    The total dataset consists of 200 samples. The new label distribution is 167 instances for label '0' (Benign) and 33 instances for label '1' (Melanoma). The train-test split preserved this distribution proportionally: the training set has 134 instances of '0' and 26 of '1', while the test set has 33 instances of '0' and 7 of '1'.
3.  **Summarize the resulting performance metrics of the classifier:**
    The Random Forest classifier achieved an overall accuracy of 0.85 on the test set. For the minority class ('1', Melanoma), the precision was 1.00, but the recall was very low at 0.14. This means that while all instances predicted as Melanoma were correct, the model missed a significant number of actual Melanoma cases. For the majority class ('0', Benign), the precision was 0.85, and the recall was 1.00.
4.  **Does significant class imbalance still exist and affect performance?**
    Yes, significant class imbalance still exists (167 Benign vs. 33 Melanoma in the full dataset, 33 Benign vs. 7 Melanoma in the test set). This imbalance severely affects the model's performance for the minority class (Melanoma), as evidenced by its very low recall of 0.14, indicating the model struggles to identify positive cases.

### Data Analysis Key Findings
*   Label extraction was successfully updated to map 'Clinical Diagnosis' '2' to Melanoma (1) and other values to Benign (0).
*   The dataset exhibits a significant class imbalance, with 167 instances labeled as Benign ($83.5\%$) and 33 instances labeled as Melanoma ($16.5\%$) out of 200 total samples.
*   The train-test split maintained this imbalance, with the test set containing 33 Benign and 7 Melanoma samples.
*   The Random Forest classifier achieved an overall accuracy of $85\%$.
*   Performance for the minority class (Melanoma) is very poor in terms of recall ($0.14$), meaning only $14\%$ of actual Melanoma cases were correctly identified by the model.
*   The precision for Melanoma is $1.00$, indicating that all cases predicted as Melanoma were indeed Melanoma.
*   Performance for the majority class (Benign) is strong, with a recall of $1.00$ and precision of $0.85$.

### Insights or Next Steps
*   Address the significant class imbalance using techniques such as oversampling (e.g., SMOTE), undersampling, or adjusting class weights within the classifier to improve the recall for the minority class (Melanoma).
*   Re-evaluate model performance using metrics more robust to class imbalance, such as the F1-score or Area Under the Receiver Operating Characteristic (AU-ROC) curve, to get a more comprehensive understanding of the model's effectiveness, especially for the critical minority class.
